In [ ]:
!pip install openai
!pip install getpass

In [ ]:
## Descrive the topic of the dataset you want to create


# Example Prompts
# prompt = "A model that takes in a puzzle-like reasoning-heavy question in English, and responds with a well-reasoned, step-by-step thought out response in Spanish."
# prompt = "A Model that takes in a question on badminton rules and responds with a well-reasoned, step-by-step thought out response."
# prompt = "A Model that takes in a question on quantum physics and responds with a well-reasoned, step-by-step thought out response."

prompt = "A Model that takes in a question on Tao Science and responds with a well-reasoned, step-by-step thought out response."

temperature = .4
number_of_examples = 100

In [ ]:
import openai
import getpass

openai.api_key = getpass.getpass("OPEN AI API KEY: ")

In [ ]:
import os
import random


def generate_example(prompt, prev_examples, temperature=.5):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 10:
            prev_examples = random.sample(prev_examples, 10)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
        max_tokens=2048,
    )

    return response.choices[0].message['content']

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print(prev_examples)

In [ ]:
def generate_system_message(prompt):

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message['content']

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

In [ ]:
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.head()

In [ ]:
# Split the data into train and test sets, with 90% in the train set
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

In [2]:
from datasets import load_dataset

# download and load all the pairs
dataset = load_dataset("facebook/flores", "all")
# dataset = load_dataset("ai4bharat/IN22-Gen", "all")

# download and load specific pairs
# dataset = load_dataset("ai4bharat/IN22-Gen", "eng_Latn-hin_Deva")

Generating gen split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [2]:
import json

# Read the JSON file as a string
with open("input.json", "r", encoding="utf-8") as f:
    json_string = f.read()

# Define the problematic Unicode escape sequence
problematic_sequence = "\\uXXXX"  # Replace XXXX with the actual problematic sequence

# Replace the problematic sequence with an empty string
cleaned_json_string = json_string.replace(problematic_sequence, "")

# Decode the cleaned JSON string
try:
    data = json.loads(cleaned_json_string)
except json.JSONDecodeError as e:
    print("Error:", e)
    # Handle the error (e.g., log, skip, or raise an exception)

Error: Invalid \uXXXX escape: line 9747 column 194 (char 6029311)


In [3]:
import json

# Specify the input and output file paths
input_file = "input.json"
output_file = "output.json"

# Read the JSON file
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Write the data back to the output file with UTF-8 encoding
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
dataset.to_json("")